In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Ashutosh Mukherjee\AppData\Local\Temp\ipykernel_7964\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


***

# FBS using pyFBS: An experimental case

> Francesco Trainotti $\,\,$ *francesco.trainotti@tum.de* <br>
> Marie Brons $\,\,$ *maribr@dtu.dk* <br>


> Advanced Structural Dynamics, DTU Copenhagen, 16-19 June 2025





***

In [2]:
import pyFBS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings("ignore")

### Data Import
Load the required predefined datasets:

In [4]:
stl_dir_A = r"./datafiles/STL/A_free.stl"
stl_dir_B = r"./datafiles/STL/B_free.stl"
stl_dir_AB = r"./datafiles/STL/AB_free.stl"

pos_xlsx = r"./datafiles/geo_pos_dir.xlsx"

df_acc_A = pd.read_excel(pos_xlsx, sheet_name='Sensors_A')
df_chn_A = pd.read_excel(pos_xlsx, sheet_name='Channels_A')
df_imp_A = pd.read_excel(pos_xlsx, sheet_name='Impacts_A')

df_acc_B = pd.read_excel(pos_xlsx, sheet_name='Sensors_B')
df_chn_B = pd.read_excel(pos_xlsx, sheet_name='Channels_B')
df_imp_B = pd.read_excel(pos_xlsx, sheet_name='Impacts_B')

df_acc_AB = pd.read_excel(pos_xlsx, sheet_name='Sensors_AB')
df_chn_AB = pd.read_excel(pos_xlsx, sheet_name='Channels_AB')
df_imp_AB = pd.read_excel(pos_xlsx, sheet_name='Impacts_AB')

## 3D view
Open 3D viewer in the background. With the 3D viewer the subplot capabilities of [PyVista](https://docs.pyvista.org/index.html) can be exploited.

In [ ]:
view3D = pyFBS.view3D(show_origin = False, show_axes = False, shape = (1,3), title = "Overview")

In [ ]:
view3D.plot.subplot(0,0)
view3D.plot.isometric_view()
view3D.plot.add_text("A structure", position='upper_left', font_size=10, color="k", font="times", name="A_structure")

view3D.add_stl(stl_dir_A,color = "#83afd2",name = "A");
view3D.show_acc(df_acc_A,scale = 1000)
view3D.show_imp(df_imp_A,scale = 1000)
view3D.show_chn(df_chn_A,scale = 1000)

In [ ]:
view3D.plot.subplot(0,1)
view3D.plot.isometric_view()
view3D.plot.add_text("B structure", position='upper_left', font_size=10, color="k", font="times", name="B_structure")

view3D.add_stl(stl_dir_B,color = "#83afd2",name = "B");
view3D.show_acc(df_acc_B,scale = 1000,overwrite = False)
view3D.show_imp(df_imp_B,scale = 1000,overwrite = False)
view3D.show_chn(df_chn_B,scale = 1000,overwrite = False)

In [ ]:
view3D.plot.subplot(0,2)
view3D.plot.isometric_view()
view3D.plot.add_text("AB structure", position='upper_left', font_size=10, color="k", font="times", name="AB_structure");

view3D.add_stl(stl_dir_AB,color = "#83afd2",name = "AB");
view3D.show_acc(df_acc_AB,scale = 1000,overwrite = False)
view3D.show_imp(df_imp_AB,scale = 1000,overwrite = False)
view3D.show_chn(df_chn_AB,scale = 1000,overwrite = False)

In [ ]:
view3D.plot.link_views()
#view3D.plot.unlink_views()

## Experimental example

## B & A+B
Load the experimental data of component B and reference assembly AB previously measured by us.

In [ ]:
YB_mea=np.load('./datafiles/EXP data FT/YB_mea.npy') # sub B data
YAB_mea=np.load('./datafiles/EXP data FT/YAB_mea.npy') # AB assembled data - the reference

YB_mea.shape, YAB_mea.shape

In [ ]:
freq = np.linspace(1, 2001, 2001)

## A
If you did not manage to get your own data, use our back up measurements, otherwise ignore the following block.

In [ ]:
YA_mea=np.load('./datafiles/EXP data FT/YA_meaBACKUP.npy') 
YA_mea.shape

df_chn_A = pd.read_excel(pos_xlsx, sheet_name='Channels_A_BACKUP')
df_imp_A = pd.read_excel(pos_xlsx, sheet_name='Impacts_A_BACKUP')


### Virtual Point Transformation
Looking at the connection-type, a VPT with rigid IDMs seems to be ideal.

Load the virtual point informations.

In [ ]:
df_vp = pd.read_excel(pos_xlsx, sheet_name='VP_Channels')
df_vpref = pd.read_excel(pos_xlsx, sheet_name='VP_RefChannels')

Create the transformation matrices:

In [ ]:
vpt_A = pyFBS.VPT(df_chn_A,df_imp_A,df_vp,df_vpref)
vpt_B = pyFBS.VPT(df_chn_B,df_imp_B,df_vp,df_vpref)

Apply the defined VP transformation on the FRFs:

In [ ]:
vpt_A.apply_VPT(freq,YA_mea)
vpt_B.apply_VPT(freq,YB_mea)

In [ ]:
plt.figure()
plt.imshow(vpt_A.Ru.astype(bool), cmap='gray', interpolation='none')
plt.colorbar()  # Optional: remove if you don't want the color scale
plt.show()

In [ ]:
plt.figure()
plt.imshow(vpt_B.Ru.astype(bool), cmap='gray', interpolation='none')
plt.colorbar()  # Optional: remove if you don't want the color scale
plt.show()

Extract the required transformed FRFs and the frequency vector:

In [ ]:
Y_A = vpt_A.vptData
Y_B = vpt_B.vptData

### Measurement quality indicators

Let's compare $\boldsymbol{u}$ with the filtered $\bar{\boldsymbol{u}}= \mathbf{R}_{\text{u}}\left( \mathbf{R}_{\text{u}} \right)^+ \boldsymbol{u}$ (and similarly for the forces) using DoF-specific and averaging criteria. For simplicity, let us check only subsystem A.

In [ ]:
vpt_A.consistency([20],[20])

In [ ]:
spec_chn = pyFBS.barchart(np.arange(1,13,1), vpt_A.specific_sensor, color='steelblue', title='Specific Channel Consistency')
spec_imp = pyFBS.barchart(np.arange(1,13,1), vpt_A.specific_impact, color='firebrick', title='Specific Impact Consistency')
spec_chn | spec_imp

In [ ]:
pyFBS.plot_coh(vpt_A.freq, vpt_A.overall_sensor, color='steelblue', title='Overall Channel Consistency')

In [ ]:
pyFBS.plot_coh(vpt_A.freq, vpt_A.overall_impact, color='firebrick', title='Overall Impact Consistency')

### LM-FBS coupling

First, construct an admittance matrix for the uncoupled system, containing substructure admittances:

$$\mathbf{Y}^\text{A|B} = \begin{bmatrix} 
\mathbf{Y}^\text{A} & \mathbf{0} \\
\mathbf{0} & \mathbf{Y}^\text{B}
\end{bmatrix}.$$

In [ ]:
Y_AnB = np.zeros((Y_A.shape[0],Y_A.shape[1]+Y_B.shape[1],Y_A.shape[2]+Y_B.shape[2]), dtype=complex)

Y_AnB[:,:Y_A.shape[1],:Y_A.shape[2]] = Y_A
Y_AnB[:,Y_A.shape[1]:,Y_A.shape[2]:] = Y_B

Next the compatibility and the equilibrium conditions has to be defined through the signed Boolean matrices ``Bu`` and ``Bf``. 

$$\mathbf{B}_\text{u}\,\boldsymbol{u} = \mathbf{0}$$
$$\boldsymbol{g} = - \mathbf{B}_\text{f}^\text{T} \boldsymbol{\lambda}$$

In [ ]:
k = 6 # number of collocated DoFs at the interface

# Define the Boolean matrix for the compatibility and equilibrium conditions. Hint: Check the order and grouping index of displacements in A and B.
Bu = np.zeros((k,Y_A.shape[1]+Y_B.shape[1]))

startIndex = 3; # 4th Column (first virtual point DOF of A)
Bu[:,startIndex:startIndex + 6] = np.identity(6)
startIndex = startIndex + 6
Bu[:,startIndex: startIndex + 6] = -np.identity(6)


# Define the Boolean matrix for the equilibrium conditions. Hint: Check the order and grouping index of forces in A and B.
Bf = np.zeros((k,Y_A.shape[2]+Y_B.shape[2]))
startIndex = 3; # 7th Column (first virtual point DOF of B)
Bf[:,startIndex:startIndex + 6] = np.identity(6)
startIndex = startIndex + 6
Bf[:,startIndex: startIndex + 6] = -np.identity(6)


In [ ]:
# visualize your Boolean matrix for verification

plt.figure()
plt.imshow(Bu)
plt.colorbar(shrink=0.5)
plt.xlabel('DoF')
plt.ylabel('Number of \n compatibility \n conditions')

In [ ]:
# visualize your Boolean matrix for verification

plt.figure()
plt.imshow(Bf)
plt.colorbar(shrink=0.5)
plt.xlabel('DoF')
plt.ylabel('Number of \n compatibility \n conditions')

For the LM-FBS method, having defined $\mathbf{Y^{\text{A|B}}}$, $\mathbf{B}_\text{u}$ and $\mathbf{B}_\text{f}$ is already sufficient to perform coupling:

In [ ]:
Y_ABn = np.zeros_like(Y_AnB,dtype = complex)

Y_ABn = Y_AnB - Y_AnB@Bf.T@np.linalg.inv(Bu@Y_AnB@Bf.T)@Bu@Y_AnB

### Results
First extract the FRFs at the reference DoFs:

In [ ]:
arr_coup_out = [0,1,2,15,16,17,18,19,20,21,22,23] # acc to keep  
arr_coup_in = [0,1,2,15,16,17,18,19,20,21,22,23,24] # force to keep
Y_AB_coupled = Y_ABn[:,arr_coup_out,:][:,:,arr_coup_in]

Y_AB_coupled.shape


The coupled and the reference results of the assembled system AB can then be compared:

In [ ]:
out = 9
inp = 4

coupled_response = Y_AB_coupled[:, out, inp]
reference_response = YAB_mea[:, out, inp]

# Plot the magnitude
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.semilogy(freq, (np.abs(coupled_response)), label='Coupled')
plt.semilogy(freq, (np.abs(reference_response)), label='Reference', linestyle='--')
plt.ylabel('Magnitude ((m/s^2)/N)')
plt.legend()
plt.grid(True)

# Plot the phase
plt.subplot(2, 1, 2)
plt.plot(freq, np.angle(coupled_response, deg=True), label='Coupled')
plt.plot(freq, np.angle(reference_response, deg=True), label='Reference', linestyle='--')
plt.ylabel('Phase (degrees)')
plt.xlabel('Frequency (Hz)')
plt.grid(True)

plt.tight_layout()
plt.show()